In [ ]:
if 'filenames' not in locals() and 'filenames' not in globals():
    SETUP_PATH = "/home/xjaros2/Documents/git/csidh-setup/"
    # filenames = [
    #      './data/parameter-search/husky/dummy/husky-clock-param-search-random-(+5)(-3)(+1)-constrained.json'
    #  ]
    filenames = ["./data/parameter-search/husky/dummy/husky-clock-param-search-random-(+5)(-3)(+1)-initial.json"]
    BINS = 50
    %cd $SETUP_PATH/notebooks

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from csidhtools import Unit
from scipy.signal import find_peaks
from matplotlib.patches import Patch

TITLE_FONTSIZE = 16
YLABEL_FONTSIZE = 13
XLABEL_FONTSIZE = 13

plt.rcParams.update({
    "text.usetex": True,
    "ytick.color" : "black",
    "xtick.color" : "black",
    "axes.labelcolor" : "black",
    "axes.edgecolor" : "black",
    "font.family" : "serif",
    "font.serif" : ["Computer Modern Serif"]
})

In [ ]:
df = None
for filename in filenames: 
    with open(filename, 'r') as f:
        current_df = pd.DataFrame( json.load(f)['measurements'])
        if df is None:
            df = current_df
        else:
            df = df.concat([df, current_df])

df["unit"] = df["unit"].apply(lambda x: Unit(repr=x))
unit_df = pd.DataFrame(df["unit"].apply(lambda u: u.__dict__()).tolist())
df = pd.concat([df.drop(columns=["unit"]), unit_df], axis=1)
df

In [ ]:
df[
    df["responses"].apply(lambda x: isinstance(x, list) and len(x) == 0) &
    (df["type"] == "CHANGING")
]

In [ ]:
# Entries where the type is changing, but there is no successful glitch (JUSTRIGHT) will be categorized as RESET
df.loc[
    df["responses"].apply(lambda x: isinstance(x, list) and len(x) == 0) &
    (df["type"] == "CHANGING"),
    "type"
] = "RESET"

In [ ]:
hue_order = ["NORMAL", "RESET", "CHANGING", "JUSTRIGHT"]
palette = ["#1E88E5", "#D81B60", "#FFC187", "#04AB8F"]

if df[df.type == "CHANGING"].empty:
    hue_order.remove("CHANGING")
    palette.remove("#FFC187")

In [ ]:
# Determine hue order with "JUSTRIGHT" 
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# First plot: 2D histogram with hue order
for i, (t,p) in enumerate(zip(hue_order, palette)):
    subset = df[df["type"] == t]
    sns.histplot(
        subset,
        x="width",
        y="offset",
        ax=axes[0],
        color=p,
        alpha=1,
        label=t,
        zorder=i,  # ensures "JUSTRIGHT" is on top
        bins=BINS
    )


axes[0].set_title("Width vs Offset by Type")
# 🧩 Custom legend
legend_elements = [Patch(facecolor=p, label=t) for (t,p) in zip(hue_order, palette)]
axes[0].legend(handles=legend_elements, title="Type")

# Second plot: histogram of "repeat" for type "JUSTRIGHT"
sns.histplot(df[df["type"] == "JUSTRIGHT"], x="repeat", ax=axes[1], bins=15)
axes[1].set_title("Repeat Histogram for JUSTRIGHT")

plt.tight_layout()
plt.show()